<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/Multi-Agent-AI-model-DnD-Character-/blob/main/multi_agent_compound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent Compound AI (DnD Character)

This project aims to create a multi-agent compound AI that can take a classic novel/story as an input, separate the story into scenes based on locations, identify any characters within the scene relevant to the plot and use this information to create a character profile and corresponding artwork. The idea is that this tool could be used to create Dungeons & Dragons campaigns based on classic novels. This would be a basic proof of concept effort in this regard.

In [1]:
#Install dependencies
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
#Login to huggingface so we can push the trained model to the hub later

from huggingface_hub import notebook_login

notebook_login()

In [3]:
#Import libraries
import torch
from transformers import pipeline

import os

## 1. Divide into Scenes

The first step is to use an agent to divide the story text into scenes based on location. In a DnD game, players progress through the story by moving from one location to the next. They can also return to locations, but that would constitute a separate scene.

In [4]:
#Maximising the space available for model loading on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF']="max_split_size_mb:512"

In [ ]:
#Load and test model (Dolly 2.0 - 7B)

generate_text = pipeline(model="databricks/dolly-v2-7b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/13.8G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
res[0]["generated_text"]

'Nuclear fission is the splitting of a nucleus into smaller nuclei. Nuclear fusion is the joining of two or more nuclei together to form a single, larger nucleus. The fusion reaction releases huge amounts of energy in the form of heat and light. Nuclear fission is used to generate electricity, while nuclear fusion is the source of the vast majority of the energy on Earth.'

In [5]:
f = open("./alice_in_wonderland_chapter1.txt", "r")

text = f.read()

prompt = f"""###Instruction:
List the locations where the plot of the provided input takes place.
For each location, provide the text of the input that takes place at that location.

###Input:
{text}

##Response:

"""

# res = generate_text(prompt)
# res[0]["generated_text"]

In [6]:
model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)
pipeline(prompt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '###Instruction:\nList the locations where the plot of the provided input takes place.\nFor each location, provide the text of the input that takes place at that location.\n\n###Input:\nAlice was beginning to get very tired of sitting by her sister\non the bank, and of having nothing to do:  once or twice she had\npeeped into the book her sister was reading, but it had no\npictures or conversations in it, `and what is the use of a book,\'\nthought Alice `without pictures or conversation?\'\n\n  So she was considering in her own mind (as well as she could,\nfor the hot day made her feel very sleepy and stupid), whether\nthe pleasure of making a daisy-chain would be worth the trouble\nof getting up and picking the daisies, when suddenly a White\nRabbit with pink eyes ran close by her.\n\n  There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!  Oh dear!  I shall be late!\'  (when s